# Актуализация показателей кредитного скорринга

Исследовать показатели заёмщиков для актуализации системы кредитного скорринга физических лиц. Определить как влияют параметры клиентов на наличие просроченной задолженности.

##  Импорт и изучение данных

In [1]:
# Импортируем необходимые библиотеки и загрузим исходные данные для анализа
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
df = pd.read_csv('/datasets/data.csv')

In [2]:
# выведем на экран первые 10 строк таблицы
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
# получим общую информацию о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Видим несоответствие типа данных в колонках `days_employed` и `total_income`  - float64, вместо int64. Данные в `days_employed` содержат как положительные, так и отрицательные значения, а также явно ошибочные значения в 5-ой строке (стаж не может быть порядка тысячи лет).

Также видим наличие неявных дубликатов в столбце `education`.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Вывод**

В каждой строке таблицы — данные о ранее выданном заемщику кредите. Часть колонок описывает заемщика: пол, возраст, образование, трудовой стаж, семейное положение, количество детей, ежемесячный доход. Остальные данные рассказывают о кредите: цель получения кредита, наличие задолженностей по возврату кредита.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски и дубликаты в данных. В колонке days_employed данные явно ошибочны - но это не страшно, т.к. они не входят в программу анализа.

Чтобы двигаться дальше, устраним проблемы в анализируемых данных.

## Предобработка данных

### Обработка пропусков

In [4]:
# подсчёт явных пропусков
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Не все пропущенные значения влияют на исследование. Так в days_employed пропуски не важны для работы. Достаточно заменить их медианными значениями характерными для анализируемых группы заещиков.

Но пропуски в total_income будут влиять на расчет зависимости между уровнем дохода и возвратом кредита в срок. Учитывая, что количество таких пропусков порядка 1% от набора данных - принебрежем данным влиянием и заменим их медианным значением.

In [5]:
# проверка отстутствия выхода за границы диаппазона отрицательных значений в 'days_employed' - в пределах возраста заемщика 
df_temp = df.query('(days_employed < 0)&(days_employed / 365 >= dob_years)')
len(df_temp)

0

Видим, что значения соответствуют интервалу возраста заемщика в днях

In [6]:
# проверка отстутствия выхода за границы диаппазона положительных значений в 'days_employed' - пределах возраста заемщика
df_temp = df.query('(days_employed > 0)&(days_employed / 365 >= dob_years)')
len(df_temp)

3445

Видим, что значения выходят за значения интервала возраста заемщика в днях. Проверим гипотезу, что значения приведены в часах.

In [7]:
# проверка положительных значений в 'days_employed' на условие значений в часах
df_temp = df.query('(days_employed > 0)&(days_employed / 365 / 24 >= dob_years)')
df_temp.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
157,0,348414.028009,38,среднее,1,женат / замужем,0,F,пенсионер,1,113560.650035,сделка с автомобилем
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
751,0,390755.464054,41,среднее,1,женат / замужем,0,M,пенсионер,0,151898.693438,операции со своей недвижимостью
776,0,365336.560325,38,среднее,1,женат / замужем,0,F,пенсионер,0,73859.425084,покупка недвижимости
1175,0,370879.508002,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1242,0,334764.259831,22,Среднее,1,Не женат / не замужем,4,F,пенсионер,0,89368.600062,получение высшего образования
1383,0,353802.811675,37,среднее,1,вдовец / вдова,2,F,пенсионер,0,216452.226085,строительство недвижимости
1446,0,338939.641634,37,СРЕДНЕЕ,1,гражданский брак,1,M,пенсионер,0,148657.128499,свадьба
1637,0,399260.448516,39,среднее,1,женат / замужем,0,M,пенсионер,0,211513.978152,операции с недвижимостью


Видим, что данные приведены не в часах - указанное количество больше чем число прожитых клиентом часов. Можно отметить, что заещики в выборке имеют статус пенсионера и безработного - проверим гипотезу о расчете стажа при нулевой дате первого трудоустройства в анкете, что при подсчете стажа может указывать на период с 01.01.1970 по дату заполнения анеты. В этом случае расхождения в стаже заемщиков в выборке могут объясняться разными датами заполнения анкеты.

In [8]:
# проверка положительных значений в 'days_employed' на условие значений в часах с даты старта UTC, на примере строки 1242
print(f'Расчетный год заполнения анкеты заемщиком index=1242: {int(1970+334764/365/24)}')

Расчетный год заполнения анкеты заемщиком index=1242: 2008


Произведенный расчет не опроверг гипотезу об отсутствии значений даты трудоустройства в анкете заемщика. Таким образом, можем принять гипотезу в качестве рабочей и произвести замену положительных значений стажа на отрицательные медианные для групп по `income_type`

In [9]:
# замена пропущенных и положительных значений в 'days_employed' на медианное значение отрицательных значенй для группировки по `income_type`
for status in df['income_type'].unique():
    median = df.query('days_employed < 0').loc[df['income_type'] == status, 'days_employed'].median()
    print(status, ': медианный стаж =', median)
    df.loc[(df['income_type'] == status)&(df['days_employed'] > 0), 'days_employed'] = median
    df.loc[(df['income_type'] == status)&(df['days_employed'].isna()), 'days_employed'] = median

сотрудник : медианный стаж = -1574.2028211070851
пенсионер : медианный стаж = nan
компаньон : медианный стаж = -1547.3822226779334
госслужащий : медианный стаж = -2689.3683533043886
безработный : медианный стаж = nan
предприниматель : медианный стаж = -520.8480834953765
студент : медианный стаж = -578.7515535382181
в декрете : медианный стаж = -3296.7599620220594


Видим, что у категорий заемщиков "пенсионер" и "безработный" отстутствует стаж. Заменим стаж для данных категорий на медианный стаж всех категорий заемщиков.

In [10]:
# замена стажа пенсионера и безработного медианным стажем всех заемщиков
for status_free in ['пенсионер', 'безработный']:
    median = df.loc[df['days_employed'] < 0, 'days_employed'].median()
    print(status_free, ': медианный стаж =', median)
    df.loc[(df['income_type'] == status_free)&(df['days_employed'] > 0), 'days_employed'] = median
    df.loc[(df['income_type'] == status_free)&(df['days_employed'].isna()), 'days_employed'] = median

пенсионер : медианный стаж = -1574.2028211070851
безработный : медианный стаж = -1574.2028211070851


In [11]:
# проверка отстутсвия положительных значений в 'days_employed' на условие значений в часах
df_temp = df.query('days_employed > 0')
len(df_temp)

0

Пропущенные значения в графе 'total_income' заменим на медианные значения дла соответствующей категории замщиков.

In [12]:
# замена пропущенных значений в 'total_income' на медианное значение для группировки по `income_type`
# df['total_income'] = df['total_income'].fillna(df['total_income'].median())
for status in df['income_type'].unique():
    median = df.loc[df['income_type'] == status, 'total_income'].median()
    df.loc[(df['income_type'] == status)&(df['total_income'].isna()), 'total_income'] = median

Заменим пропущенные значения в 'total_income' на медианное значение для группировки по `income_type` + 'education' + 'dob_years'.

In [13]:
# составим сводную таблицу для расчета пропущенных значений в колонке `total_income`
svod_3 = pd.pivot_table(df, index=['income_type', 'education', 'dob_years'], aggfunc={'total_income':'median'})
svod_3

total_income
income_type education      dob_years               
безработный Высшее         45         202722.511368
            среднее        31          59956.991984
в декрете   СРЕДНЕЕ        39          53829.130729
госслужащий ВЫСШЕЕ         21         349229.424952
                           24         175195.751491
...                                             ...
сотрудник   среднее        71         154130.628685
                           74          98945.906177
            ученая степень 37         115949.039788
                           58         268411.214536
студент     высшее         22          98201.625314

[1226 rows x 1 columns]

In [15]:
# проверка отстуствия пропусков
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [16]:
b

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

In [61]:
# подсчет количества выявленных заемщиков с нулевым стажем
df[df['dob_years'] == 0]['dob_years'].count()

101

In [77]:
# данный показатель не повлияет существенно на результат исследования,
# поэтому заменим значения 0 в 'dob_years' на медианное значение показателя для группировки по `income_type`
for status in df['income_type'].unique():
    median = df.loc[(df['income_type'] == status)&(df['dob_years'] != 0), 'dob_years'].median()
    df.loc[(df['income_type'] == status)&(df['dob_years'] == 0), 'dob_years'] = median
    
df[df['dob_years'] == 0]['dob_years'].count()

0

**Вывод**

Все явные и неявные пропуски были выявлены и заменены медианными и средними значениями. Пропуски данных в `days_employed` вероятно обусловлены отстутствием данных о трудоустройстве у пенсионеров и безработных замещиков. Пропуски данных в `total_income` вероятно обусловленых отстутствием дохода в анкете, т.к. данные замщики относятся к безработным и пенсионерам.

### Изменение типов данных

Приведем данные по стажу, возрасту и доходу заемщика к целым числам - данной точности достаточно для анализа.

In [81]:
# изменение типа данных методом astype() в `total_income` на целочисленный и `days_employed` на целочисленный по модулю
df['total_income'] = df['total_income'].astype('int64')
df['days_employed'] = df['days_employed'].astype('int64').abs()
df['dob_years'] = df['dob_years'].astype('int64')

In [82]:
# проверка изменения типа данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 15 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
purpose_extract     21454 non-null object
income_type_id      21454 non-null int64
total_income_q      21454 non-null category
dtypes: category(1), int64(8), object(6)
memory usage: 2.3+ MB


**Вывод**

Вещественный тип данных заменен на целочисленный по модулю значения. Приведение к модулю потребовалось в связи с вероятной  ошибкой в расчете стажа - из меньшей даты была вычтена большая.

### Обработка дубликатов

Проверим и удалим явные дубликаты.

In [23]:
# подсчет явных дубликатов
df.duplicated().sum()

54

In [24]:
# приведение данных в `education` к нижнему регистру
df['education'] = df['education'].str.lower()

In [25]:
# удаление явных дубликатов сторок методом drop_duplicates()
df = df.drop_duplicates().reset_index(drop=True)

In [26]:
# проверка отсутствия явных дубликатов
df.duplicated().sum()

0

Проверим и удалим неявные дубликаты.

In [27]:
# проверка неявных дубликатов в `education` через проверку уникальных значений
df['education'].sort_values().unique()

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

In [28]:
# проверка отсутствия неявных дубликатов в `education` через проверку уникальных значений
df['education'].sort_values().unique()

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

In [29]:
# проверка неявных дубликатов в `family_status` через проверку уникальных значений
df['family_status'].sort_values().unique()

array(['Не женат / не замужем', 'в разводе', 'вдовец / вдова',
       'гражданский брак', 'женат / замужем'], dtype=object)

In [30]:
# приведение формата значений в `family_status` к нижнему регистру
df['family_status'] = df['family_status'].str.lower()

In [31]:
# проверка неявных дубликатов в `income_type` через проверку уникальных значений
df['income_type'].sort_values().unique()


array(['безработный', 'в декрете', 'госслужащий', 'компаньон',
       'пенсионер', 'предприниматель', 'сотрудник', 'студент'],
      dtype=object)

In [32]:
# проверка неявных дубликатов в `children` через проверку уникальных значений
df['children'].sort_values().unique()

array([-1,  0,  1,  2,  3,  4,  5, 20])

In [33]:
# удаление неявных дубликатов в `children` через удаление вероятной опечатки в виде знака "-" 
df['children'] = df['children'].replace(-1, 1)

In [34]:
# проверка удаления неявных дубликатов в `children` через проверку уникальных значений
df['children'].sort_values().unique()

array([ 0,  1,  2,  3,  4,  5, 20])

Проверим уникальные значения в столбце 'purpose'

In [35]:
purpose_uniq = df['purpose'].unique()
purpose_uniq

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Вывод**

Явные и неявные дубликаты выявлены и устранены. Возможная причина неявных дубликатов - ошибки ручного набора значений.
Также присутствует большое количество неявных дубликатов в столбце purpose. Для их устранения выделим леммы в значениях столбца.

### Лемматизация

In [36]:
lemma_list = []

for aim in purpose_uniq:
    lemma = m.lemmatize(aim)
    lemma_list.append(lemma)
lemma_list

[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['жилье', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['автомобиль'

In [37]:
# составим список уникальных слов, хотя бы одно из которых присутствует в каждом списке `lemma_list`
purpose_lemma = ['жилье', 'недвижимость', 'автомобиль', 'образование', 'свадьба']

Добавим столбец 'purpose_extract' с найденными уникальными словами, характеризующими цель кредита

In [38]:
# функция подбора уникального значения
def purpose_extract(purpose):
    purpose_lemmas =  ' '.join(m.lemmatize(purpose))
    if 'жилье' in purpose_lemmas:
        return 'жилье'
    if  'недвижимость' in purpose_lemmas:
        return 'недвижимость'
    if  'автомобиль' in purpose_lemmas:
        return 'автомобиль'
    if  'образование' in purpose_lemmas:
        return 'образование'
    if  'свадьба' in purpose_lemmas:
        return 'свадьба'  
#     for aim_lemma in purpose_lemma:
#         if aim_lemma in purpose_lemmas:
#             return aim_lemma

In [39]:
df['purpose_extract'] = df['purpose'].apply(purpose_extract)

In [40]:
# проверка добавления столбца
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_extract
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,1574,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье
6,0,2879,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье
7,0,152,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье


In [41]:
# объединим синонимы целей кредита 'жилье' и 'недвижимость', заменив на 'недвижимость'
df['purpose_extract'] = df['purpose_extract'].replace(to_replace='жилье', value='недвижимость')

In [42]:
# проверка замены знанчения  жилье' на 'недвижимость' в 'purpose_extract'
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_extract
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,1574,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,152,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


**Вывод**

Выделены 4 унифицированные цели кредитования для всех заемщиков - недвижимость, автомобиль, образование, свадьба.

### Категоризация данных

Выделим отельные словари для наборов данных с одинаковыми значениями

In [43]:
# выделим словарь для `education`
education_dict = df[['education_id', 'education']] 
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [44]:
# выделим словарь для `family_status`
family_status_dict = df[['family_status_id', 'family_status']] 
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


In [46]:
# выделим словарь для `income_type`
income_type_unique = df['income_type'].unique()

In [47]:
# вычисление значения для 'income_type_id'
def income_type_number(row):
    for i in range(len(income_type_unique)):
        if row['income_type'] == income_type_unique[i]:
            return i

In [48]:
# добавим колонку с индексом 'income_type_id' для 'income_type'
df['income_type_id'] = df.apply(income_type_number, axis=1)

In [49]:
# выделим словарь для 'income_type'
income_type_dict = df[['income_type_id', 'income_type']] 
income_type_dict = income_type_dict.drop_duplicates().reset_index(drop=True)
income_type_dict

,income_type_id,income_type
0,0,сотрудник
1,1,пенсионер
2,2,компаньон
3,3,госслужащий
4,4,безработный
5,5,предприниматель
6,6,студент
7,7,в декрете


Категоризируем колонку 'total_income', разделив на квартили.

In [50]:
# проведем оценку распределения на квартили, оценим статистические показатели распределения 'total_income'
df['total_income'].describe().apply("{0:.0f}".format)

count      21454
mean      165320
std        98187
min        20667
25%       107623
50%       142594
75%       195820
max      2265604
Name: total_income, dtype: object

In [51]:
# для расределения значений 'total_income' на квартили используем метод qcut
label = ['1_квартиль', '2_квартиль', '3_квартиль', '4_квартиль']
df['total_income_q'] = pd.qcut(df['total_income'], 4, labels=label)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_extract,income_type_id,total_income_q
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,0,4_квартиль
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,0,2_квартиль
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,0,3_квартиль
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,0,4_квартиль
4,0,1574,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,1,3_квартиль


**Вывод**

Категоризация данных проведена через выделение словарей для колонок `education`, `family_status`, `income_type`. Категоризация `total_income` проведена через разделение заемщиков на квартили по уровню дохода.

## Проверка завистимостей

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [53]:
# рассчет рейнтинга по проценту невозврата в зависимости от количества детей
debt_reiting_children = df.groupby('children').agg({'debt':['mean', 'count']})
debt_reiting_children = debt_reiting_children.rename(columns={'debt':'Сводная таблица'}, level=0)
debt_reiting_children = debt_reiting_children.rename(columns={'mean':'% невозврата', 'count':'Количество заемщиков'}, level=1)
debt_reiting_children.index = debt_reiting_children.index.rename('Количество детей')
debt_reiting_children.style.format({('Сводная таблица', '% невозврата'):'{:.2%}'})

In [54]:
nevozvr = (debt_reiting_children['Сводная таблица']['% невозврата'] * debt_reiting_children['Сводная таблица']['Количество заемщиков']).sum() / \
debt_reiting_children['Сводная таблица']['Количество заемщиков'].sum()
f'Средневзвешенный процент невозврата = {nevozvr:.2%}'

'Средневзвешенный процент невозврата = 8.12%'

**Вывод**

Видим, что доля невозвратов по кредитом среди заемщиков без детей ниже, чем у замщиков имеющих детей. Возможно это связано с дополнительными расходами на детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [56]:
# рассчет рейнтинга по проценту невозврата в зависимости от семейного положения
debt_reiting_family_status = df.groupby('family_status').agg({'debt':['mean', 'count']})
debt_reiting_family_status = debt_reiting_family_status.rename(columns={'debt':'Сводная таблица'}, level=0)
debt_reiting_family_status = debt_reiting_family_status.rename(columns={'mean':'% невозврата', 'count':'Количество заемщиков'}, level=1)
debt_reiting_family_status.index = debt_reiting_family_status.index.rename('Семейное положение')
debt_reiting_family_status.style.format({('Сводная таблица', '% невозврата'):'{:.2%}'})

**Вывод**

Анализ показывает, что максимальная доля невозвратных кредитов приходится на заещиков не состоящих в браке или состоящих в гражданском браке. Возможно это связано с отсутствием у данных категорий заемщиков навыков ответственности и способности к управлению личными финансами.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [58]:
# рассчет рейнтинга по проценту невозврата в зависимости от семейного положения
debt_reiting_total_income_q = df.groupby('total_income_q').agg({'debt':['mean', 'count']})
debt_reiting_total_income_q = debt_reiting_total_income_q.rename(columns={'debt':'Сводная таблица'}, level=0)
debt_reiting_total_income_q = debt_reiting_total_income_q.rename(columns={'mean':'% невозврата', 'count':'Количество заемщиков'}, level=1)
debt_reiting_total_income_q.index = debt_reiting_total_income_q.index.rename('Уровень доходов')
debt_reiting_total_income_q.style.format({('Сводная таблица', '% невозврата'):'{:.2%}'})

**Вывод**

Анализ показал, что наименьший уровень невозврата зафиксирован у заемшиков с доходами 4 и 1 квартиля. Возможно это связано с более развитыми навыками управления личными финансами у данных категорий заемщиков.

- Как разные цели кредита влияют на его возврат в срок?

In [59]:
# рассчет рейнтинга по проценту невозврата в зависимости от семейного положения
debt_reiting_purpose_extract = df.groupby('purpose_extract').agg({'debt':['mean', 'count']})
debt_reiting_purpose_extract = debt_reiting_purpose_extract.rename(columns={'debt':'Сводная таблица'}, level=0)
debt_reiting_purpose_extract = debt_reiting_purpose_extract.rename(columns={'mean':'% невозврата', 'count':'Количество заемщиков'}, level=1)
debt_reiting_purpose_extract.index = debt_reiting_purpose_extract.index.rename('Цель кредита')
debt_reiting_purpose_extract.style.format({('Сводная таблица', '% невозврата'):'{:.2%}'})

**Вывод**

Анализ показывает, что наименьшая доля невозвратных кредитов приходится на кредиты связанные с недвижимостью. Возможно это обусловлено наличием достаточного обеспечения в виде ипотеки. Кредиты на свадьбу также имеют небольшой процент невозврата, что возможно обусловлено меньшей суммой и сроком кредитования по сравнению с кредитами на другие цели, а также возможными поручительствами со стороны родителей.

## Общий вывод

Проведенное исследование надежности заемщиков показало, что семейное положение и наличие детей у замещиков в среднем влияют на процент невозврата по кредитам. При этом:

- наличие супруга/супруги или наличие таких в прошлом снижает вероятность невозврата по кредитам до 6,6% -7,6%
- отсутствие детей снижает вероятность невозврата до 7,5%, наличие детей у замщика повышает риск невозврата кредита и требует более детального анализа платежеспособности замещика.

Также была выявлен более низкий процент невозврата у заещиков с доходами относящимися к 1 и 4 квартилю (8,0% и 7,1% соответственно) против более высконого процента невозврата замещиками с средним уровнем доходов, относящихся к 2 и 3 квартилю (8,5% и 8,9% соответственно).

По целям кредитования: наименьший процент невозврата приходитя на заемщиков кредитующихся для приобретения недвижимости (7,2%) и для проведения свадьбы (8%), наибольший процент невозврата у заемщиков кредитующихся для покупки автомобиля (9,4%) или берущих образовательный кредит(9,2%).

Можно рекомендовать включить указанные показатели в систему кредитного скорринга банка.